#  1. Recommendation - Loading

In [1]:
import os
import zipfile
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse import vstack
import pickle
from pathlib import Path

c:\Users\riccardoricci\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%run "0.Splitting_Reading_Recommendation_File.ipynb"

In [3]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [4]:
def loading_recommendations(df):
    # keep only users that do recommend, eliminate those who does not recommend
    df = df[df["is_recommended"]==True]
    df["is_recommended"] = df["is_recommended"].apply(lambda x: 1 if x == True else 0)
    df['user_id_categorical'] = pd.Categorical(df['user_id']).codes
    ## App_ID starts from 1. I leave the 0 to the user_id column. See later
    df['app_id_categorical'] = pd.Categorical(df['app_id']).codes + 1
    df = df.sort_values(by="user_id_categorical")
    df["date"] = pd.to_datetime(df["date"])
    return df

In [5]:
recommendations = loading_recommendations(recommendations_df)
recommendations

C:\Users\riccardoricci\AppData\Local\Temp\ipykernel_1908\4069270214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_recommended"] = df["is_recommended"].apply(lambda x: 1 if x == True else 0)
C:\Users\riccardoricci\AppData\Local\Temp\ipykernel_1908\4069270214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_id_categorical'] = pd.Categorical(df['user_id']).codes
C:\Users\riccardoricci\AppData\Local\Temp\ipykernel_1908\4069270214.py:7: SettingWithCopyWarning: 
A value is trying to be se

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,user_id_categorical,app_id_categorical
30478477,235540,28,2,2015-10-17,1,21.9,0,30478477,0,1698
24437527,49520,35,16,2015-08-12,1,2.1,0,24437527,0,895
34510797,627690,8,0,2022-03-06,1,103.3,0,34510797,0,12333
22036352,1454400,6,2,2022-02-18,1,762.6,0,22036352,0,28682
36809055,317400,15,0,2015-08-01,1,18.0,0,36809055,0,3676
...,...,...,...,...,...,...,...,...,...,...
5228835,397540,0,0,2020-11-25,1,160.9,14306059,5228835,12663129,6218
39735230,1112830,0,0,2020-10-10,1,1.0,14306060,39735230,12663130,22680
23151356,1407200,0,0,2022-09-02,1,171.0,14306061,23151356,12663131,27909
25486974,1987080,0,0,2022-08-28,1,15.5,14306062,25486974,12663132,35826


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=78c133f5-defd-458d-ba8f-cbdc9ae58cfb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>